In [54]:
import os                       # for working with files
import numpy as np              # for numerical computationss
import pandas as pd             # for working with dataframes
import matplotlib.pyplot as plt # for plotting informations on graph and images using tensors
import json
from PIL import Image
from typing import List
import io
import sys
from pydantic import BaseModel
from tensorflow.keras.models import load_model
import torch                    # Pytorch module 
from torch.utils.data import DataLoader # for dataloaders
import torchvision.transforms as transforms   # for transforming images into tensors 
from torchvision.utils import make_grid       # for data checking
from torchvision.datasets import ImageFolder  # for working with classes and images
from torchsummary import summary              # for getting the summary of our model
import torch.nn as nn
import torch.nn.functional as F
from fastapi import FastAPI, File, UploadFile, HTTPException
from colabcode import ColabCode
import pickle
from fastapi.middleware.cors import CORSMiddleware
%matplotlib inline

In [55]:
!pip install flask_restful
!pip install colabcode
!pip install pydantic
!pip install fastapi
!pip install python-multipart

In [56]:
with open('crop_dict.json') as json_file:
    data1 = json.load(json_file)

In [57]:
app = FastAPI()
#api = Api(app)

In [58]:
def ConvBlock(in_channels, out_channels, pool=False):
    layers = [nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
             nn.BatchNorm2d(out_channels),
             nn.ReLU(inplace=True)]
    if pool:
        layers.append(nn.MaxPool2d(4))
    return nn.Sequential(*layers)


# Model Architecture
class ResNet9(nn.Module):
    def __init__(self, in_channels, num_diseases):
        super().__init__()
        
        self.conv1 = ConvBlock(in_channels, 64)
        self.conv2 = ConvBlock(64, 128, pool=True) # out_dim : 128 x 64 x 64 
        self.res1 = nn.Sequential(ConvBlock(128, 128), ConvBlock(128, 128))
        
        self.conv3 = ConvBlock(128, 256, pool=True) # out_dim : 256 x 16 x 16
        self.conv4 = ConvBlock(256, 512, pool=True) # out_dim : 512 x 4 x 44
        self.res2 = nn.Sequential(ConvBlock(512, 512), ConvBlock(512, 512))
        
        self.classifier = nn.Sequential(nn.MaxPool2d(4),
                                       nn.Flatten(),
                                       nn.Linear(512, num_diseases))
        
    def forward(self, xb): # xb is the loaded batch
        out = self.conv1(xb)
        out = self.conv2(out)
        out = self.res1(out) + out
        out = self.conv3(out)
        out = self.conv4(out)
        out = self.res2(out) + out
        out = self.classifier(out)
        return out

In [59]:
PATH = 'plant-disease-model-complete.pth'
disease_model = torch.load(PATH,map_location ='cpu')
#model = pickle.load(open('cropped_model.pkl','rb'))

In [60]:
def predict_image(img, model=disease_model):
    """
    Transforms image to tensor and predicts disease label
    :params: image
    :return: prediction (string)
    """
    yb = model(img)
    # Pick index with highest probability
    _, preds = torch.max(yb, dim=1)
    prediction = disease_classes[preds[0].item()]
    # Retrieve the class label
    return prediction

In [61]:
disease_classes = ['Apple___Apple_scab',
                   'Apple___Black_rot',
                   'Apple___Cedar_apple_rust',
                   'Apple___healthy',
                   'Blueberry___healthy',
                   'Cherry_(including_sour)___Powdery_mildew',
                   'Cherry_(including_sour)___healthy',
                   'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot',
                   'Corn_(maize)___Common_rust_',
                   'Corn_(maize)___Northern_Leaf_Blight',
                   'Corn_(maize)___healthy',
                   'Grape___Black_rot',
                   'Grape___Esca_(Black_Measles)',
                   'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)',
                   'Grape___healthy',
                   'Orange___Haunglongbing_(Citrus_greening)',
                   'Peach___Bacterial_spot',
                   'Peach___healthy',
                   'Pepper,_bell___Bacterial_spot',
                   'Pepper,_bell___healthy',
                   'Potato___Early_blight',
                   'Potato___Late_blight',
                   'Potato___healthy',
                   'Raspberry___healthy',
                   'Soybean___healthy',
                   'Squash___Powdery_mildew',
                   'Strawberry___Leaf_scorch',
                   'Strawberry___healthy',
                   'Tomato___Bacterial_spot',
                   'Tomato___Early_blight',
                   'Tomato___Late_blight',
                   'Tomato___Leaf_Mold',
                   'Tomato___Septoria_leaf_spot',
                   'Tomato___Spider_mites Two-spotted_spider_mite',
                   'Tomato___Target_Spot',
                   'Tomato___Tomato_Yellow_Leaf_Curl_Virus',
                   'Tomato___Tomato_mosaic_virus',
                   'Tomato___healthy']
PATH = 'plant-disease-model-complete.pth'

In [62]:
transform = transforms.Compose([
        transforms.Resize(256),
        transforms.ToTensor(),
    ])

class Crop(BaseModel):
    N: float 
    K: float 
    P: float 
    temperature: float 
    humidity: float 
    ph: float 
    rainfall: float 
    class Config:
        schema_extra = {
            "example": {
                "N": 93, 
                "K": 56, 
                "P": 42,
                "temperature": 23.857240,
                "humidity": 82.225730,
                "ph": 7.382763,
                "rainfall": 195.094831
            }
        }

class Prediction(BaseModel):
  filename: str
  contenttype: str
  prediction: str
# Define the /prediction route


In [63]:
@app.on_event("startup")
def load_model():
    global model
    model = pickle.load(open("cropped_model.pkl", "rb"))

@app.get('/')
def index():
    return {'message': 'This is the homepage of the API '}


@app.post('/cropprediction')
def get_crop_prediction(data: Crop):
    received = data.dict()
    N = received['N']
    K = received['K']
    P = received['P']
    temperature = received['temperature']
    humidity = received['humidity']
    ph = received['ph']
    rainfall = received['rainfall']
    pred_name = model.predict([[N, K, P,
                                temperature, humidity, ph, rainfall]]).tolist()[0]
    #print(str(pred_name))
    return {'prediction': data1[str(pred_name)]}

@app.post('/cropdiseaseprediction/', response_model=Prediction)
async def prediction_route(file: UploadFile = File(...)):
    contents = await file.read()
    image = Image.open(io.BytesIO(contents))
    img_t = transform(image)
    img_u = torch.unsqueeze(img_t, 0)
    prediction = predict_image(img_u,disease_model)
    return {
      'filename': file.filename,
      'contenttype': file.content_type,
      'prediction': prediction
    }


In [64]:
origins = [
    "http://localhost",
    "http://localhost:8080",
    "http://localhost:3002",
]
app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

In [65]:

server = ColabCode(port=10000, code=False)

In [66]:
server.run_app(app=app)

Public URL: NgrokTunnel: "https://e769-35-231-147-163.ngrok.io" -> "http://localhost:10000"


INFO:     Started server process [467]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:10000 (Press CTRL+C to quit)


INFO:     27.7.22.252:0 - "GET / HTTP/1.1" 200 OK
INFO:     27.7.22.252:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     27.7.22.252:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     27.7.22.252:0 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     27.7.22.252:0 - "POST /cropprediction HTTP/1.1" 200 OK
INFO:     27.7.22.252:0 - "POST /cropdiseaseprediction/ HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [467]
